In [1]:
!pip install transformers -U


In [2]:
import pandas as pd


In [3]:
data = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")
data.head()

<ipython-input-3-dfd017dc2c35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")


,Abstract,Question,Answer,Label,TemporalAspect
0,"Wallace, 38, called Gastonia home from the age...",How often does Brewer talk about the legal sys...,pushkin liked to speak at high noon usually,yes,Frequency
1,He tried and tried to please every person who ...,What time of day was it?,5 times a century,yes,Frequency
2,"One day, as Mary was picking the blueberries, ...",Can you always see them from the same location?,3:00 AM,yes,Typical Time
3,"The exhibition's range, while very varied, rem...",Is Hall still serving?,when asked,yes,Typical Time
4,Pushkin published his first poem at the age of...,How many times a week does the average scienti...,about once a century,yes,Event Duration


In [4]:
data['Label'].value_counts()


no     2826
yes    2174
Name: Label, dtype: int64

In [5]:
data = data[['Abstract','Question','Answer','Label']]

In [6]:
data


,Abstract,Question,Answer,Label
0,"Wallace, 38, called Gastonia home from the age...",How often does Brewer talk about the legal sys...,pushkin liked to speak at high noon usually,yes
1,He tried and tried to please every person who ...,What time of day was it?,5 times a century,yes
2,"One day, as Mary was picking the blueberries, ...",Can you always see them from the same location?,3:00 AM,yes
3,"The exhibition's range, while very varied, rem...",Is Hall still serving?,when asked,yes
4,Pushkin published his first poem at the age of...,How many times a week does the average scienti...,about once a century,yes
...,...,...,...,...
4995,"Like all energy, gravitational potential energ...",How long does the dermis take to heal after a ...,for 30 weeks,no
4996,Deborah likes to add some sugar with the lettu...,Is the French state still being centralized?,twice a decade,no
4997,"A. DeConcini Federal Courthouse, 405 W. Congre...",When did Frank's father become an alcoholic?,yes they do it every month for fun,no
4998,"Search ""Emiliano Malaquez"" and you'll find he'...",Was he still being introduced an hour later?,about once a century,no


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
# from transformers import AutoTokenizer, ElectraForQuestionAnswering
import torch
from transformers import AutoTokenizer, ElectraForSequenceClassification

In [8]:
# tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/electra-base-squad2")
# model = ElectraForQuestionAnswering.from_pretrained("bhadresh-savani/electra-base-squad2")
tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/electra-base-emotion")
model = ElectraForSequenceClassification.from_pretrained("bhadresh-savani/electra-base-emotion")

In [9]:
model = model.to('cuda')


In [10]:
tokenizer

ElectraTokenizerFast(name_or_path='bhadresh-savani/electra-base-emotion', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [11]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=768, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

In [12]:
import numpy as np  # Import numpy for NaN handling

# Map string labels to numerical values
label_mapping = {"yes": 1, "no": 0}
data["Label"] = data["Label"].map(label_mapping)

data["Abstract"] = data["Abstract"].fillna("")
data["Question"] = data["Question"].fillna("")
data["Answer"] = data["Answer"].fillna("")

# Concatenate the strings for context, question, and answer
X = ["[CLS] " + str(abstract) + " [SEP] " + str(question) + " [SEP] " + str(answer) + " [SEP]" for abstract, question, answer in zip(data["Abstract"], data["Question"], data["Answer"])]

y = list(data["Label"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenizing a single sequence (context, question, answer concatenated)
X_train_tokenized = tokenizer(
    X_train,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",  # Assuming you're using PyTorch
)

X_val_tokenized = tokenizer(
    X_val,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",  # Assuming you're using PyTorch
)



In [13]:
X_train_tokenized.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
print(X_train_tokenized['attention_mask'][0])


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0])


In [15]:
len(X_train),len(X_val)


(4000, 1000)

In [16]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [17]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [18]:
train_dataset[5]


<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([  101,   101,  1996,  2217,  1997, 28935,  4226,  2480,  1005,  1055,
         20463,  2435,  2126,  2000,  7487,  1037,  2665, 23041,  2243,  3236,
          1999,  1037, 20682,  1997,  5024,  2250,  1012,   102,  2129,  2411,
          2515,  2002,  4685,  1999,  2694,  3065,  1029,   102,  2296,  1022,
          3134,   102,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [19]:

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [20]:
!pip install optuna

In [21]:
import optuna
from transformers import Trainer, TrainingArguments
import torch



def objective(trial):
    num_epochs = trial.suggest_int('num_epochs', 3, 10)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    args = TrainingArguments(
        output_dir="output",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate

    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    try:
        trainer.train()
        result = trainer.evaluate()
        return result['eval_loss']
    except RuntimeError as e:
        print(f"Trial {trial.number} failed with CUDA out-of-memory error. Trying with different parameters.")
        torch.cuda.empty_cache()
        return float('inf')

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

best_params = study.best_params


[I 2023-12-20 08:38:08,816] A new study created in memory with name: no-name-eb439f66-fc5e-48e3-82ff-74f13418a564
<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.726600
1000,0.685200


<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


[I 2023-12-20 08:39:26,065] Trial 0 finished with value: 0.6975715160369873 and parameters: {'num_epochs': 4, 'batch_size': 16, 'learning_rate': 2.5568020197193734e-05}. Best is trial 0 with value: 0.6975715160369873.


<class 'transformers.trainer_utils.EvalPrediction'>


...

<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.723000
1000,0.694300
1500,0.690100
2000,0.692000
2500,0.689300
3000,0.685400
3500,0.684600
4000,0.690100
4500,0.684100
5000,0.686900


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1500 already exists and is non-empty.Saving will

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-12-20 08:53:49,052] Trial 4 finished with value: 0.6852396130561829 and parameters: {'num_epochs': 10, 'batch_size': 4, 'learning_rate': 0.00020359245614829792}. Best is trial 4 with value: 0.6852396130561829.


<class 'transformers.trainer_utils.EvalPrediction'>


In [22]:
import json

# Save best_params to a JSON file
with open('best_hyperparameters(Electra).json', 'w') as file:
    json.dump(best_params, file)

In [ ]:
# Define Trainer
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.675900
1000,0.659700
1500,0.631500
2000,0.587000
2500,0.548600
3000,0.495100
3500,0.454800
4000,0.404200
4500,0.366000
5000,0.334900


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1500 already exists and is non-empty.Saving will

TrainOutput(global_step=5000, training_loss=0.5157810119628906, metrics={'train_runtime': 263.2728, 'train_samples_per_second': 151.934, 'train_steps_per_second': 18.992, 'total_flos': 184724511840000.0, 'train_loss': 0.5157810119628906, 'epoch': 10.0})

In [ ]:
trainer.evaluate()


<ipython-input-16-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.5694762468338013,
 'eval_accuracy': 0.509,
 'eval_precision': 0.4292929292929293,
 'eval_recall': 0.39080459770114945,
 'eval_f1': 0.40914560770156444,
 'eval_runtime': 1.9202,
 'eval_samples_per_second': 520.772,
 'eval_steps_per_second': 65.097,
 'epoch': 10.0}

In [ ]:
np.set_printoptions(suppress=True)


In [ ]:
# text = "That was good point"
context = "Wallace, 38, called Gastonia home from the age of 8 until she graduated from Hunter Huss High School in 1983.	"
question = "How often does Brewer talk about the legal system marketplace?	"
answer = "pushkin liked to speak at high noon usually	"
input_sequence = f"[CLS] {context} [SEP] {question} [SEP] {answer} [SEP]"

# text = "go to hell"
tokenized_input = tokenizer(input_sequence,padding = True, truncation = True, return_tensors='pt').to('cuda')
# Make predictions
with torch.no_grad():
    outputs = model(**tokenized_input)

# Get the predicted label
logits = outputs.logits
predicted_label = "yes" if logits.argmax() == 1 else "no"
print("Predicted Label:", predicted_label)

# outputs = model(**inputs)
# print(outputs)
# predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# print(predictions)
# predictions = predictions.cpu().detach().numpy()
# predictions

Predicted Label: yes


In [ ]:
trainer.save_model('CustomModel')


In [22]:
! pip install accelerate -U
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
os._exit(00)